In [1]:
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
import tensorflow as tf
from tensorflow.keras import layers, models


## Preprocess data

Download the full dataset from [here](https://www.kaggle.com/datasets/hassiahk/stanford-cars-dataset-full) and place it in the same directory as this file.

In [3]:
# Setup directories

!mkdir data/
!mkdir data/images/
!unzip archive.zip  # change the directory/filename of the zip if needed
!mv images/ data/car_ims/


mkdir: data/: File exists
mkdir: data/images/: File exists
unzip:  cannot find or open archive.zip, archive.zip.zip or archive.zip.ZIP.
mv: rename images/ to data/car_ims/images/: No such file or directory


If you are on Colab, download `cars_annos.mat` from [here](https://www.kaggle.com/datasets/jessicali9530/stanford-cars-dataset) and upload it to `data/`

In [2]:
# Load annotations

data_path = Path() / 'data'
margin = 16
im_path = data_path / 'images'
im_path.mkdir(exist_ok=True)

mat = sio.loadmat(data_path / 'cars_annos.mat')


# Class names
class_names = [
    name[0].replace('/', '_')
    for name in mat['class_names'][0]
]
num_classes = len(class_names)
print(f"{num_classes} class names loaded")  # There should be 196 class names


In [ ]:
# This processes image files and places them in a separate directory
# You only have to run it once for each runtime

image_count = 0
for [fp], [[x1]], [[y1]], [[x2]], [[y2]], [[class_id]], _ in mat['annotations'][0]:
    fp = Path(fp)
    src_img = cv2.imread(str(data_path / fp))
    h, w = src_img.shape[:2]
    class_name = class_names[class_id - 1]
    # Crop image
    x1 = max(0, x1 - margin)
    y1 = max(0, y1 - margin)
    x2 = min(x2 + margin, w)
    y2 = min(y2 + margin, h)
    dst_img = src_img[y1:y2, x1:x2]
    # Save image
    image_dir = im_path / class_name
    image_dir.mkdir(exist_ok=True)
    cv2.imwrite(str(image_dir / fp.name), dst_img)
    image_count += 1
    if image_count % 1000 == 0:
        print(f"progress: {image_count}")

print(f"Total {image_count} images loaded")  # There should be 16186 images


## Load data

In [ ]:
# Create datasets
# Change the parameters as you wish

train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
    str(im_path),
    validation_split=0.1,
    image_size=(256, 256),
    batch_size=64,
    label_mode='categorical',
    subset='both',
    seed=42,
)
batch = next(train_ds.as_numpy_iterator())
input_shape = batch[0][0].shape

# `train_ds` and `val_ds` contain both X and Y values
print(train_ds, val_ds)
print(f"Input shape: {input_shape}")


In [ ]:
# Visualize the dataset

plt.figure(figsize=(16, 16))
for i in range(16):
    plt.subplot(4, 4, i + 1)
    plt.imshow(batch[0][i].astype(int))
    plt.xticks([])
    plt.yticks([])


## Build model

In [11]:
# Build your model. Good luck!
# TODO
